In [2]:
#Kyle Krakoski

import tweepy

%run C:\Users\kitko\Downloads\twitter_credentials.py
    
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)
api = tweepy.API(auth)


In [7]:
import datetime
from datetime import timedelta
from datetime import date
import pandas as pd
import numpy as np

def animal_tweet_count(animal= '', state = '', city = '', parks = '', filters = '', count_retweets = False, 
                       location = '', start_year = '', start_month = '', start_day = '', num_days = 7, group = 1, num_needed = 1000): 
    
    api = tweepy.API(auth)
    #Initialize values and ensure relevant variables are strings
    tweet_list = [] 
    animal_count_list = [] 
    animal = str(animal)
    state = str(state)
    city = str(city)
    parks =  str(parks)
    filters = str(filters) 
    location = str(location)
    
    if count_retweets:  #Filters out retweets if set to False
        retweets = '' 
    else:
        retweets = '-RT'
        
    search_terms = (animal, state, city, parks, filters, retweets) #Concates together all the search and filter terms   
    last_id = -1 # id of last tweet seen
    
    start_date = datetime.date(start_year, start_month,start_day) #Sets the first day to start counting tweets
    number_of_days = timedelta(days = num_days ) #Gets how many days to count
    end_date = start_date + number_of_days #Sets ending date for counting tweets
    incr_days = timedelta(days = group) #Used to increment the date by the given number of days  
    until_date = start_date + incr_days #Sets the "until" date, which is a day after the "since" date

    while start_date < end_date: #Iterates and gets counts for each day until the start date equals the end date
    
        while len(tweet_list) < num_needed: #Searches over "num_needed" tweets instead of just 100 
            try:
                new_tweets = api.search(q = " ".join(search_terms), count = 100, geocode= location, since = start_date, 
                                        until = start_date + incr_days,  max_id = str(last_id - 1))
            except tweepy.TweepError as e:
                print("Error", e)
                break
            else:
                if not new_tweets:
                   # print("Could not find any more tweets!")
                    break
                tweet_list.extend(new_tweets)             
                last_id = new_tweets[-1].id
                 
        animal_count= len(tweet_list) #Counts how many tweets fit the search parameter for the given day       
        animal_count_list.append(animal_count) #Adds the tweet count for the given day to a list of tweet counts
        start_date = start_date + incr_days #Changes the start date to the next day
        
        tweet_list = [] #Resets the tweet_list to empty for the next search
        last_id = -1 #Resets the last_id to 
        
    return(animal_count_list) #Returns a list of all the counts
                    
animal_tweet_count(animal = 'deer', state = '', city = '', parks = '',  
                   filters = '-%deer%valley  -%deer%creek  -%Promo%Code', 
                   count_retweets = False, location = '38.9972,-105.5478,380km', 
                   start_year = 2020, start_month = 12, start_day = 14, num_days = 1, num_needed = 1000)


#Function Parameters:
#animal - Input the animal you  want to search for
#state- Input the state name you want to search tweets for.  Mainly used if unsure of location coordinates, 
    #since location coordinates returned better results
#city - Input the city name you want to search tweets for.
#parks - Input any parks or reserves you'd like to search tweets for
#filters- Don't count the tweet if any of these terms are in it- used if a certain day is an extreme outlier; 
    #could be getting tweets for an unrelated event.  The current filter terms caused some days to return an abnormally
    #high count. 
#count_retweets- If false, retweets aren't counted 
#location- Used for the geocode parameter in the search function.  Inputs are latitude, longitude,and a radius 
    #from that point.  Returns all tweets within this radius if geotagged, if not, checks tweeter's profile info 
    #and returns tweet if profile is within radius
#start_year, start_month, start_day- Year, month, day of starting date
#num_days- How many days of counts you want
#group- How many days to aggregate together for a count
#num_needed- Max number of tweets to return


[29]

In [3]:
def convert_to_csv(day_counts):

    count_df = pd.DataFrame(data = day_counts,
                           index = ['Day {}'.format(ii) for ii in  range(1, len(day_counts)+1)]) #Makes a dataframe from the animal list count
    count_df_T = count_df.T
    print(count_df_T)
    count_df_T.to_csv(r'C:\Users\kitko\OneDrive\Documents\xdaydataframe.csv', index = False)
    
convert_to_csv(day_counts = animal_tweet_count(animal = 'deer', state = '', city = '', parks = '',  
                   filters = '-%deer%valley  -%deer%creek  -%Promo%Code', 
                   count_retweets = False, location = '38.9972,-105.5478,380km', 
                   start_year = 2020, start_month = 12, start_day = 9, num_days = 7, group = 1, num_needed = 1000))

   Day 1  Day 2  Day 3  Day 4  Day 5  Day 6  Day 7
0     38     24     32     14     34     29     33


In [10]:
def get_tweet_text():  #A function to return tweet text for every tweet matching the search parameters.
    num_needed2 = 1000
    tweet_list2 = []
    last_id2 = -1 # id of last tweet seen
    while len(tweet_list2) < num_needed2:
        try:
            new_tweets2 = api.search(q='deer -%deer%valley  -%deer%creek  -%Promo%Code -RT', count = 100, geocode = '38.9972,-105.5478,380km', since = '2020-12-11', until = '2020-12-15',  max_id = str(last_id2 - 1))
        except tweepy.TweepError as e:
            print("Error", e)
            break
        else:
            if not new_tweets2:
                print("Could not find any more tweets!")
                break
            tweet_list2.extend(new_tweets2)
            last_id2 = new_tweets2[-1].id
    print(len(tweet_list2))
    return([tweet.text for tweet in tweet_list2])
get_tweet_text()

Could not find any more tweets!
109


['I’m pretty sure I just saw a woman walking her pet deer.',
 'Beyond Rubicon drew four tags in their favorite deer unit and they have deer meat on their minds. However, Brian an… https://t.co/SclOW93kik',
 '@bigbushelsag Looks like a no eyed deer to me.... like. Those guys have no idea(r) what they’re doing.',
 '@blueberrydurr Mule deer are cute AF',
 '@heinyboii Right? Just a deer in headlights 🥴',
 "@StevenRinella is hunting just south of the border in Sonora, Mexico. If you want to find venison down here, you'd… https://t.co/k0C39Iuf2q",
 "@Truthfully_Mike Deer hunter since I've never seen it",
 'Good morning, lovelies. Nothing to see here, just a deer walking a dog ☺ https://t.co/8phSKrEPhs',
 'For the #Warrior in your life, consider this #HandCarved Deer Antler #AxePendant. Visit our Etsy store to save 10%… https://t.co/cz4pW3anw8',
 '@joni_deer Beautiful, cute, gay.',
 '@BuggJayd "You\'re welcome~\nThe white deer is very beautiful and goes well with snow, so it\'s a Christmas pr